In [193]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, size
from pyspark.sql.types import BooleanType, IntegerType, ArrayType, StringType
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.clustering import LDA
from nltk.stem import RSLPStemmer

import re
import nltk
import spacy
import numpy as np

In [194]:
spark = SparkSession.builder \
    .appName("SPARK TRAB - QUESTION 1 D") \
    .getOrCreate()

In [195]:
df_orig = spark.read.option("header", "false").option("delimiter", "\t").csv("./data/debate-tweets.tsv")

## Objetivo

In [196]:
df = df_orig.select("_c0", "_c1")

df = df.withColumnRenamed("_c0", "id") \
                     .withColumnRenamed("_c1", "content")

df.show()

+------------------+--------------------+
|                id|             content|
+------------------+--------------------+
|522394422710136832|@anacddd verdade,...|
|522394422806581248|              Que ñ*|
|522394422731100160| Vou quebrar a Bruna|
|522394422810783745|agora vou p segun...|
|522394423137943553|Me sinto tão bem ...|
|522394423188271104|Eu estou aqui, de...|
|522394423238606848|Quando vai embora...|
|522394423528022016|@paynecaralhudo k...|
|522394423632875521|Conceição da Barr...|
|522394424010362881| @Maniavato te amo ♥|
|522394424048091138|Alg me curtindo rs ♡|
|522394424010358784|@MiiluAA No, porq...|
|522394423741906944|#EMABiggestFansJu...|
|522394424568213505|@raizabatista dev...|
|522394424920506368|Me senti ate d fe...|
|522394424811458560|qual o sentido de...|
|522394425029574656|I'm at Lava Rápid...|
|522394425121841153|Fica comentando m...|
|522394425461579777|"odeio que me man...|
|522394425960701952|CAMAMTEBABILONFRA...|
+------------------+--------------

In [197]:
def extract_dilma(text):
    if re.search(r'\bDilma\b', text, re.IGNORECASE):
        return True
    else:
        return False
    
def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

In [198]:
extract_dilma_udf = udf(extract_dilma, BooleanType())

df = df.withColumn("hasDilma", extract_dilma_udf(df["content"]))

df_dilma = df.filter(col("hasDilma")).select("content")

df_dilma.show()

+--------------------+
|             content|
+--------------------+
|"""@marinaecologi...|
|Pensando seriamen...|
|O gigante acordou...|
|@KennedyAncar @Dr...|
|Quem vota na Dilm...|
|OLHA DILMA MINHA ...|
|@BLOGdoPEPE mas N...|
|"Dilma, viado, co...|
|"""@MirandaSa_: K...|
|Aprendi com a Dil...|
|Dilma funciona co...|
|"""@emirsader: ""...|
|Lembrar dos 13,00...|
|@LordePagao pedim...|
|Só eu acho qe o A...|
|@paulaingridd que...|
|e falam q nao vao...|
|#AecioPelaMudanca...|
|Bora pensar Dilma...|
|Acho que o pessoa...|
+--------------------+


In [199]:
remove_punctuation_udf = udf(remove_punctuation, StringType())

df_dilma = df_dilma.withColumn("content_clean", remove_punctuation_udf(col("content")))

df_dilma.show()

+--------------------+--------------------+
|             content|       content_clean|
+--------------------+--------------------+
|"""@marinaecologi...|marinaecologia Di...|
|Pensando seriamen...|Pensando seriamen...|
|O gigante acordou...|O gigante acordou...|
|@KennedyAncar @Dr...|KennedyAncar Drim...|
|Quem vota na Dilm...|Quem vota na Dilm...|
|OLHA DILMA MINHA ...|OLHA DILMA MINHA ...|
|@BLOGdoPEPE mas N...|BLOGdoPEPE mas No...|
|"Dilma, viado, co...|Dilma viado conti...|
|"""@MirandaSa_: K...|MirandaSa_ KKKKK ...|
|Aprendi com a Dil...|Aprendi com a Dil...|
|Dilma funciona co...|Dilma funciona co...|
|"""@emirsader: ""...|emirsader Meu vot...|
|Lembrar dos 13,00...|Lembrar dos 1300 ...|
|@LordePagao pedim...|LordePagao pedimo...|
|Só eu acho qe o A...|Só eu acho qe o A...|
|@paulaingridd que...|paulaingridd quer...|
|e falam q nao vao...|e falam q nao vao...|
|#AecioPelaMudanca...|AecioPelaMudanca ...|
|Bora pensar Dilma...|Bora pensar Dilma...|
|Acho que o pessoa...|Acho que o

In [200]:
tokenizer = Tokenizer(inputCol="content_clean", outputCol="words")
df_tokens = tokenizer.transform(df_dilma)

df_tokens.show()

+--------------------+--------------------+--------------------+
|             content|       content_clean|               words|
+--------------------+--------------------+--------------------+
|"""@marinaecologi...|marinaecologia Di...|[marinaecologia, ...|
|Pensando seriamen...|Pensando seriamen...|[pensando, seriam...|
|O gigante acordou...|O gigante acordou...|[o, gigante, acor...|
|@KennedyAncar @Dr...|KennedyAncar Drim...|[kennedyancar, dr...|
|Quem vota na Dilm...|Quem vota na Dilm...|[quem, vota, na, ...|
|OLHA DILMA MINHA ...|OLHA DILMA MINHA ...|[olha, dilma, min...|
|@BLOGdoPEPE mas N...|BLOGdoPEPE mas No...|[blogdopepe, mas,...|
|"Dilma, viado, co...|Dilma viado conti...|[dilma, viado, co...|
|"""@MirandaSa_: K...|MirandaSa_ KKKKK ...|[mirandasa_, kkkk...|
|Aprendi com a Dil...|Aprendi com a Dil...|[aprendi, com, a,...|
|Dilma funciona co...|Dilma funciona co...|[dilma, funciona,...|
|"""@emirsader: ""...|emirsader Meu vot...|[emirsader, meu, ...|
|Lembrar dos 13,00...|Lem

In [201]:
stopwordList = nltk.corpus.stopwords.words('portuguese')

remover = StopWordsRemover(inputCol="words", outputCol="filtered", stopWords = stopwordList)
df_filtered_tokens = remover.transform(df_tokens)

df_filtered_tokens.show()

+--------------------+--------------------+--------------------+--------------------+
|             content|       content_clean|               words|            filtered|
+--------------------+--------------------+--------------------+--------------------+
|"""@marinaecologi...|marinaecologia Di...|[marinaecologia, ...|[marinaecologia, ...|
|Pensando seriamen...|Pensando seriamen...|[pensando, seriam...|[pensando, seriam...|
|O gigante acordou...|O gigante acordou...|[o, gigante, acor...|[gigante, acordou...|
|@KennedyAncar @Dr...|KennedyAncar Drim...|[kennedyancar, dr...|[kennedyancar, dr...|
|Quem vota na Dilm...|Quem vota na Dilm...|[quem, vota, na, ...|[vota, dilma, mer...|
|OLHA DILMA MINHA ...|OLHA DILMA MINHA ...|[olha, dilma, min...|[olha, dilma, fil...|
|@BLOGdoPEPE mas N...|BLOGdoPEPE mas No...|[blogdopepe, mas,...|[blogdopepe, nobl...|
|"Dilma, viado, co...|Dilma viado conti...|[dilma, viado, co...|[dilma, viado, co...|
|"""@MirandaSa_: K...|MirandaSa_ KKKKK ...|[mirandasa_

In [202]:
vectorizer = CountVectorizer(inputCol="filtered", outputCol="features")
vectorizer_model = vectorizer.fit(df_filtered_tokens)

df_features = vectorizer_model.transform(df_filtered_tokens)

In [203]:
lda = LDA(k=5, maxIter=10, featuresCol="features")
lda_model = lda.fit(df_features)

# Tópicos identificados pelo LDA
topics = lda_model.describeTopics()
vocab = vectorizer_model.vocabulary

In [204]:
def topic_words(topic, vocab):
    return [vocab[idx] for idx in topic.termIndices]

In [205]:
topics_rdd = topics.rdd.map(lambda row: topic_words(row, vocab))
topics_words = topics_rdd.collect()

for idx, topic in enumerate(topics_words):
    print(f"Topic {idx}: {', '.join(topic)}")

Topic 0: dilma, aécio, debatenosbt, ta, brasil, falando, n, pra, p, debate
Topic 1: dilma, cara, debate, aécio, , chorar, pra, bloco, mal, debatenosbt
Topic 2: dilma, aécio, debatenarecord, ta, , debate, pra, faz, cara, agora
Topic 3: dilma, , aécio, pra, vai, q, debate, aecio, cara, ta
Topic 4: dilma, q, , pra, ta, governo, falar, sbt, aécio, pq


In [206]:
def get_main_topic_udf(topicDist):
    return int(np.argmax(topicDist))

In [207]:
get_main_topic = udf(get_main_topic_udf, IntegerType())
transformed = lda_model.transform(df_features).withColumn("mainTopic", get_main_topic(col("topicDistribution")))

In [208]:
# Mostra as primeiras 10 sentenças do primeiro tópico (por exemplo)
top_sentences = transformed.filter(col("mainTopic") == 0).select("content").take(10)

print("Sentenças Principais do Tópico 0 Relacionadas à Palavra 'Dilma':")
for i, sentence in enumerate(top_sentences):
    print(f"{i+1}: {sentence['content']}")

Sentenças Principais do Tópico 0 Relacionadas à Palavra 'Dilma':
1: Entre Dilma e Aécio, prefiro entregar o Brasil pros índios e pedir desculpas 👌
2: Votei na enquete: Dilma jantou Aécio no #DebatenaBand #MelhorcomDilma13 RT @UOL http://t.co/NnOv0UY3r0 #Eleições2014 http://t.co/8eQNdd5OeI""
3: Dilma ou Aécio? #mudabrasil #vamosfazeradiferenca #vempraurna #todomundo #selfieservico #issomudaojogo… http://t.co/xAGdk2z2u7
4: Entre Dilma e Aecio eu prefiro devolver o Brasil pros índios e pedir desculpas.
5: Entre a Dilma ou o Aécio, prefiro devolver o Brasil para os índios e pedir desculpas!!!--'
6: Em São Leopoldo vencemos com @tarsogenro  e @dilma. Seguimos dialogando com a população para ampliar o apoio! http://t.co/O5YfgOKWn2
7: """@leandrofortes: TCE retirou contas do site citado por Dilma, denuncia Minas Sem Censura https://t.co/YKOu6kZ3pG"""
8: Ataques Dilma×Aécio   😂😂😂😂😂
9: Eleitorado decidido: Aécio 42%, Dilma 42%.
10: “@leandrofortes: TCE retirou contas do site citado por Dilma, d

In [209]:
# Mostra as primeiras 10 sentenças do primeiro tópico (por exemplo)
top_sentences = transformed.filter(col("mainTopic") == 1).select("content").take(10)

print("Sentenças Principais do Tópico 1 Relacionadas à Palavra 'Dilma':")
for i, sentence in enumerate(top_sentences):
    print(f"{i+1}: {sentence['content']}")

Sentenças Principais do Tópico 1 Relacionadas à Palavra 'Dilma':
1: """@MirandaSa_: KKKKK RT @CellyMattosI: http://t.co/pbrHKaHYq0"" kkkkkk...olha ai Dilma"
2: Aprendi com a Dilma que 13-4=7
3: #UnaFinalMuyReñida: El duelo final entre Dilma y Aécio http://t.co/oJG8FegobN vía @el_pais
4: IBOPE en las ultimas semanas había pronosticado que Dilma perdía en ballotage con M. Silva (la cual salió 3ra). Son un jugador político mas
5: @elizondogabriel endorsements in Brazil have little value. Exceptions: Lula-Dilma and Aecio-Anastasia in Minas.
6: """@Carlosmperalta: El duelo final entre Dilma y Aécio http://t.co/inGtu84cc0 vía @el_pais"" /muy revelador de la contienda electoral."
7: "??????????----&gt;""@UOL: Aprovação do governo Dilma sobe e atinge 40%, indica pesquisa http://t.co/JzzqJoXWD1 @UOL"""
8: Luego de nuestro triunfo, estamos en vigilia del de #Dilma en #Brasil. Les dejo una nota que me hicieron @comunicasul http://t.co/SsjfNWX75I
9: MAH OÊ! RT @paaahlu: @Ocharadista Me manda o vid

In [210]:
# Mostra as primeiras 10 sentenças do primeiro tópico (por exemplo)
top_sentences = transformed.filter(col("mainTopic") == 2).select("content").take(10)

print("Sentenças Principais do Tópico 2 Relacionadas à Palavra 'Dilma':")
for i, sentence in enumerate(top_sentences):
    print(f"{i+1}: {sentence['content']}")

Sentenças Principais do Tópico 2 Relacionadas à Palavra 'Dilma':
1: @KennedyAncar @Drimone @jornaldaREALMENTE DILMA CAGUEJOU,ESTAVA NERVOSA,SEM PREPARO,MUITO REPETITIVA VOLTA MUITO NO PASSADO,AÉCIO FOI MELHOR
2: OLHA DILMA MINHA FILHA ENTENDA FILHINHA VOCÊ JÁ ERA NÃO RESISTE TOMA MEU LENÇO: #45Confirma ! !!! !!! !!!
3: @BLOGdoPEPE mas Noblat reconhece superioridade de Dilma.
4: @renatorabelobr @Dra_SimoneQUE PROPOSTA,DILMA SÓ ATACOU E MUITA REPETITIVA  BOLTS DO PASSADO
5: Até você reconhece que Dilma e um desastre
6: "@Danilo_Figueira @carlosmaltz O processamento mental de Dilma é em cima da decoreba passada pelo ""maqueteiro"" da campanha"
7: Kkkkkkk vejam Dilma vs Aécio no parafernalha kkkkkkkkk
8: @Marconi45Aecio @teresinhalopes @MadameWilde @Melloalmeida45 @BlogOlhoNaMira Ponto eletrônico disfarçado de brinco foi única inovação Dilma
9: Para quem está criticando a oratória da Dilma, informo que Hitler falava muito, muito bem. ;)
10: Vereador Nabil, eu retifico seu entendimento sobr

In [211]:
# Mostra as primeiras 10 sentenças do primeiro tópico (por exemplo)
top_sentences = transformed.filter(col("mainTopic") == 3).select("content").take(10)

print("Sentenças Principais do Tópico 3 Relacionadas à Palavra 'Dilma':")
for i, sentence in enumerate(top_sentences):
    print(f"{i+1}: {sentence['content']}")

Sentenças Principais do Tópico 3 Relacionadas à Palavra 'Dilma':
1: """@marinaecologia: Dilma não arruma nem os dentes, imagina o país."""
2: Pensando seriamente em chamar o manicômio pra quem quer votar na Dilma
3: O gigante acordou tanto que vão votar na Dilma de novo 😂😂😂
4: Quem vota na Dilma merece apanhar por que pqp hein
5: "Dilma, viado, continua que tá bonito! ""Não, não me abandone. Não me desespere, porque eu não posso ficar sem você ♪"""
6: Dilma funciona com internet discada.
7: """@emirsader: ""Meu voto é a favor do Brasil, meu voto é Dilma"", diz Emicida http://t.co/k54fv7RIcW""@emicida #orgulhodevocêgaroto"
8: Lembrar dos 13,00 reais, lembrem Dilma 13, 13,00 reais
9: @LordePagao pedimos alegando que não iam mandar mais porque a Dilma não ganhou aqui no Sul.
10: Só eu acho qe o Aécio sambou na cara da Dilma ontem ? #DebateDaBand


In [212]:
# Mostra as primeiras 10 sentenças do primeiro tópico (por exemplo)
top_sentences = transformed.filter(col("mainTopic") == 4).select("content").take(10)

print("Sentenças Principais do Tópico 4 Relacionadas à Palavra 'Dilma':")
for i, sentence in enumerate(top_sentences):
    print(f"{i+1}: {sentence['content']}")

Sentenças Principais do Tópico 4 Relacionadas à Palavra 'Dilma':
1: SOU MELO SOU AMAZONENSE SOU 90. A HUMILDADE VENCE A ARROGÂNCIA E PREPOTÊNCIA A DILMA E O EDUARDO SÃO MUITO PARECIDOS (MENTEM)
2: "todos concordamos com a Dilma,né? Ela mesmo disse '' precisamos mudar'', portanto, #aeciopelamudança"" @Gusttavo_lima #Goiânia!"""
3: """@giworldfacts: @DaniloGentili DERROTAS DE DILMA em São Paulo http://t.co/wIteYs7Tgi"""
4: @fecorreianv tentativa fail de se sobressair, Dilma suja.
5: Tico Santa Cruz &gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;Dilma e Aécio http://t.co/d8uP9ClOYF
6: Brilha Dilma. Reaças piram http://t.co/lJrjNcFpMI
7: @DaniContin Menas Dilma, menas !
8: @Emiliano_Jose @lopesfernandeEU VI UMA DILMA,DESCONTROLADA,MUITO NERVOSA,DESPREPARADA,GAGUEJANDO REPETITIVA,E VOLTANDO SEMPRE EM 2000.!!!!!!
9: @_dacor @dannixiu @michele_gsantos @camargonatty @CaGomees alo, Dilma? Pedeu a hora hoje? 👂☎
10: Estamos pendientes de la defensa de la integración regional antiimperialista #Dilma. Aqu

In [213]:
# tive um resultado, mas não sei se foi interessante, por enquanto vou manter assim
# lemmatizer deu resultado pior q o esperado, então resolvi remover e deixar sem